# Class 10

In [5]:
struct MCDP_without_optimization
    u::Vector{Real}
    β::Real
    P::Matrix{Real}
end

In [6]:
job_search_problem = MCDP_without_optimization([1.0, 0.3], 1-0.05/12, [0.9 0.1; 0.03 0.97])

MCDP_without_optimization(Real[1.0, 0.3], 0.9958333333333333, Real[0.9 0.1; 0.03 0.97])

In [7]:
function iterate_value_function(v::Vector{T}, problem::MCDP_without_optimization) where T <: Real
    u, β, P = problem.u, problem.β, problem.P
    return u .+ β*P*v
end

iterate_value_function (generic function with 1 method)

In [8]:
iterate_value_function([0.0, 0.0], job_search_problem)

2-element Vector{Float64}:
 1.0
 0.3

In [9]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
v = zeros(2)
for t=1:maxiter
    new_v = iterate_value_function(v, job_search_problem)
    distance = sum((v .- new_v).^2)
    v = new_v
    if distance < tolerance 
        numiter = t
        break
    end
end
v

2-element Vector{Float64}:
 114.63042102531075
 109.39188033307501

In [10]:
numiter

1554

In [11]:
using LinearAlgebra: I
function solve(problem::MCDP_without_optimization)
    return (I - problem.β*problem.P) \ problem.u
end

solve (generic function with 1 method)

In [29]:
A = [1 0; 0 1]
b = [1, 2]
A \ b
# solution to Ax = b
# iterative solution?

2-element Vector{Float64}:
 1.0
 2.0

In [31]:
AA = [1 0 1; 0 1 2]
x = AA \ b
AA * x
# minimize (Ax - b)'(Ax - b)

2-element Vector{Float64}:
 1.0000000000000002
 2.000000000000001

In [33]:
AAA = [1 0; 0 1; 1 2]
bb = [1, 2, 3]
x = AAA \ bb
AAA * x
# this is only a projection

3-element Vector{Float64}:
 0.6666666666666659
 1.3333333333333335
 3.333333333333333

In [40]:
P_large = Array{Float64}(undef, 1_000, 1_000)
fill!(P_large, 1/1000)
u = ones(1_000, 1)

1000×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [43]:
@time inv(I - 0.95*P_large) * u

  0.248226 seconds (10 allocations: 23.392 MiB, 56.36% gc time)


1000×1 Matrix{Float64}:
 20.00000000000002
 20.00000000000003
 20.00000000000003
 20.00000000000002
 20.00000000000002
 20.000000000000025
 20.00000000000002
 20.00000000000002
 20.00000000000002
 20.000000000000025
 20.00000000000003
 20.00000000000003
 20.00000000000002
  ⋮
 20.00000000000003
 20.000000000000036
 20.000000000000036
 20.00000000000004
 20.00000000000003
 20.000000000000043
 20.000000000000036
 20.000000000000036
 20.000000000000036
 20.000000000000036
 20.000000000000032
 20.000000000000032

In [44]:
@time (I - 0.95*P_large) \ u

  0.083586 seconds (8 allocations: 22.904 MiB)


1000×1 Matrix{Float64}:
 20.00000000000003
 20.000000000000078
 19.999999999999904
 20.000000000000043
 19.99999999999999
 20.00000000000003
 20.000000000000007
 20.000000000000064
 19.999999999999996
 20.000000000000046
 20.000000000000004
 19.999999999999968
 19.99999999999998
  ⋮
 20.00000000000003
 20.000000000000032
 20.00000000000002
 20.000000000000036
 20.00000000000001
 20.000000000000014
 20.000000000000014
 20.000000000000025
 20.000000000000014
 20.000000000000018
 20.000000000000018
 20.000000000000014

In [45]:
@show A

A = [1 0; 0 1]


2×2 Matrix{Int64}:
 1  0
 0  1

In [39]:
solve(job_search_problem)

2-element Vector{Float64}:
 114.79887745556647
 109.56033676333075

This is not needed:

~struct MCDP_with_discrete_choices
    choices::Vector{MCDP_without_optimization}
end~

In [13]:
low_intensity  = MCDP_without_optimization([1.0, 0.3], 0.95, [0.9 0.1; 0.01 0.99])
high_intensity = MCDP_without_optimization([1.0, 0.1], 0.95, [0.9 0.1; 0.3 0.7])
endogenous_job_search = [low_intensity, high_intensity]

2-element Vector{MCDP_without_optimization}:
 MCDP_without_optimization(Real[1.0, 0.3], 0.95, Real[0.9 0.1; 0.01 0.99])
 MCDP_without_optimization(Real[1.0, 0.1], 0.95, Real[0.9 0.1; 0.3 0.7])

In [14]:
function argmax_bellman(choices::Vector{MCDP_without_optimization}, v::Vector{T})::Vector{Int32} where T<:Real
    N = length(choices)
    K = length(choices[1].u)
    argmax = zeros(Int32, K)
    
    for k = 1:K
        best_index = 0
        best_value = -1e+16
        for n = 1:N
            candidate_value = choices[n].u + choices[n].β * choices[n].P * v
            if candidate_value[k] > best_value
                best_index = n
                best_value = candidate_value[k]
            end
        end
        argmax[k] = best_index
    end
    return argmax
end

argmax_bellman (generic function with 1 method)

In [15]:
function iterate_value_function(v::Vector{T}, choices::Vector{MCDP_without_optimization}) where T<:Real
    N = length(choices)
    K = length(choices[1].u)
    
    c = argmax_bellman(choices, v)
    
    new_v = zeros(K)
    for k = 1:K
        n = c[k]
        RHS = choices[n].u + choices[n].β * choices[n].P * v
        new_v[k] = RHS[k]
    end
    return new_v
end


iterate_value_function (generic function with 2 methods)

In [16]:
iterate_value_function([100.0, 0.0], endogenous_job_search)

2-element Vector{Float64}:
 86.5
 28.599999999999998

In [17]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
v = zeros(2)
for t=1:maxiter
    new_v = iterate_value_function(v, endogenous_job_search)
    distance = sum((v .- new_v).^2)
    v = new_v
    if distance < tolerance 
        numiter = t
        break
    end
end
v

2-element Vector{Float64}:
 16.010234453917704
 13.917211198103743

# Policy function iteration

In [18]:
function iterate_policy_function(c::Vector{T}, choices::Vector{MCDP_without_optimization})::Vector{T} where T<:Integer
    u = similar(choices[1].u)
    P = similar(choices[1].P)
    β = choices[1].β
    
    for k = 1:length(c)
        choice = choices[c[k]]
        # for each choice c[k], what is the flow utility and the transition probs?
        u[k] = choice.u[k]
        P[k, :] .= choice.P[k, :]
    end
    # then solve for the value of this choice
    v = solve(MCDP_without_optimization(u, β, P))
    return argmax_bellman(choices, v)
end

iterate_policy_function (generic function with 1 method)

In [19]:
iterate_policy_function(ones(Int32, 2), endogenous_job_search)

2-element Vector{Int32}:
 1
 2

In [20]:
tolerance = 1e-6
maxiter = 100000

numiter = 1
c = ones(Int32, 2)
for t=1:maxiter
    new_c = iterate_policy_function(c, endogenous_job_search)
    distance = sum((c .- new_c).^2)
    c = new_c
    if distance < tolerance 
        numiter = t
        break
    end
end
c

2-element Vector{Int32}:
 1
 2

In [21]:
A = ones(1_000, 1_000)
""

""

In [22]:
B = similar(A)
""

""

In [23]:
C = zeros(size(A))
""

""

In [26]:
D = Array{Float64}(undef, 2,2)

2×2 Matrix{Float64}:
 1.43e-322  2.18523e-314
 3.0e-323   2.18523e-314